In [2]:
from os.path import join as pjoin
from ms.utils.ms_utils import get_project_path
import os
import pandas as pd

In [3]:
path_to_features = pjoin(get_project_path(), "resources", "tabzilla", "preprocessed_test")
features_files_names = os.listdir(path_to_features)
features_files = [pjoin(path_to_features, i) for i in os.listdir(path_to_features)]
path_to_target = pjoin(get_project_path(), "resources", "tabzilla")
target_folders = ["absquantile2", "absquantile3", "absuniform2", "absuniform3", "rel_xgboost_resnet", "target_diff"]
target_files = []
for folder in target_folders:
    target_files.append(pjoin(path_to_target, f"{folder}_test", "metrics.csv"))

In [4]:
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
import xgboost as xgb

lr = {
    "name": "LogReg",
    "model": LogisticRegression(penalty='l2', solver="lbfgs", max_iter=100),
    "params": {
        "penalty": ["l2", "none"],
        "C": [0.01, 0.1, 1, 10],
        "solver": ["newton-cholesky", "lbfgs", "sag", "saga"],
    }
}

mlp = {
    "name": "MLP",
    "model": MLPClassifier(
        hidden_layer_sizes=(50,),
        activation="relu",
        solver="adam",
        alpha=0.0001,
        batch_size="auto",
        learning_rate="constant",
        learning_rate_init=0.001,
        max_iter=200,
    ),
    "params": {
        "hidden_layer_sizes": [(10,), (50,), (100,)],
        "activation": ["logistic", "tanh", "relu"],
        #"solver": ["lbfgs", "sgd", "adam"],
        "alpha": [0.0001, 0.001, 0.01],
        #"batch_size": ["auto", 100, 200, 300],
        "learning_rate": ["constant", "invscaling", "adaptive"],
        "learning_rate_init": [0.001, 0.01, 0.1],
        #"max_iter": [200, 400, 600],
    },
}

xgb = {
    "name": "XGB",
    "model": xgb.XGBClassifier(),
    "params": { 
        'max_depth': [3, 5, 7, 9],
        'learning_rate': [0.01, 0.1, 0.3],
        'n_estimators': [5, 10, 50, 100],
        "eval_metric" : ["merror", "mlogloss"], 
    }
}

metamodels = [lr, mlp, xgb]

In [16]:
all_dfs = []
for i, feature_file in enumerate(features_files):
    feature_folder_name = features_files_names[i][:-4]
    for j, target_file in enumerate(target_files):
        target_folder_name = target_folders[j]
        for metamodel in metamodels:
            df = pd.read_csv(
                pjoin(get_project_path(), "resources", "tabzilla", f"{target_folder_name}_test", feature_folder_name, f"{metamodel['name']}_results.csv"), index_col=0
            )
            mean_df = pd.DataFrame(df.mean(),).T#columns=[f"{feature_folder_name[13:]}_{target_folder_name}_{metamodel['name']}"])
            index = pd.MultiIndex.from_arrays([
                [feature_folder_name[13:]],
                [target_folder_name],
                [metamodel['name']]
            ], names=["preproc", "target", "metamodel"])
            mean_df.index = index
            all_dfs.append(mean_df)

In [17]:
all_dfs[0]

,,,fit_time_mean,score_time_mean,test_b_acc_mean,test_f1_mean,test_prec_mean,test_rec_mean,test_roc_mean,fit_time_std,score_time_std,test_b_acc_std,test_f1_std,test_prec_std,test_rec_std,test_roc_std
preproc,target,metamodel,,,,,,,,,,,,,,
minmax,absquantile2,LogReg,0.017902,0.011845,0.439286,0.42688,0.43273,0.44,0.439286,0.002116,0.001482,0.107651,0.111921,0.117983,0.107643,0.107651


In [21]:
all_dfs = pd.concat(all_dfs, axis=0)
all_dfs.drop(["fit_time_mean", "score_time_mean"], axis=1, inplace=True)

In [30]:
all_dfs.sort_values(by="test_b_acc_mean", ascending=False, inplace=False)

test_b_acc_mean  test_f1_mean  \
preproc  target       metamodel                                  
standard absquantile2 XGB               0.541667      0.528826   
minmax   absquantile2 XGB               0.535863      0.519185   
standard absquantile2 LogReg            0.522917      0.511288   
                      MLP               0.522024      0.509914   
outliers absquantile2 XGB               0.520833      0.504433   
...                                          ...           ...   
power    absquantile3 LogReg            0.312778      0.295897   
outliers target_diff  LogReg            0.303810      0.357320   
power    absquantile3 MLP               0.298056      0.281746   
standard absquantile3 MLP               0.295833      0.290510   
outliers target_diff  XGB               0.294782      0.398815   

                                 test_prec_mean  test_rec_mean  test_roc_mean  \
preproc  target       metamodel                                                 
standard absquantile2 XGB              0.548504       0.541905       0.541667   
minmax   absquantile2 XGB              0.545787       0.535873       0.535863   
standard absquantile2 LogReg           0.527518       0.521587       0.522917   
                      MLP              0.523345       0.521032       0.522024   
outliers absquantile2 XGB              0.527025       0.520238       0.520833   
...                                         ...            ...            ...   
power    absquantile3 LogReg           0.306661       0.312381            NaN   
outliers target_diff  LogReg           0.318816       0.430000       0.427083   
power    absquantile3 MLP              0.295715       0.298571            NaN   
standard absquantile3 MLP              0.308677       0.296111            NaN   
outliers target_diff  XGB              0.409875       0.422857       0.583333   

                                 fit_time_std  score_time_std  test_b_acc_std  \
preproc  target       metamodel                                                 
standard absquantile2 XGB            0.000661        0.000661        0.108572   
minmax   absquantile2 XGB            0.062866        0.011318        0.130516   
standard absquantile2 LogReg         0.000718        0.000724        0.120422   
                      MLP            0.004182        0.000517        0.125623   
outliers absquantile2 XGB            0.001853        0.001497        0.119629   
...                                       ...             ...             ...   
power    absquantile3 LogReg         0.014529        0.002473        0.094160   
outliers target_diff  LogReg         0.005290        0.000699        0.131794   
power    absquantile3 MLP            0.007013        0.000675        0.102489   
standard absquantile3 MLP            0.003333        0.000528        0.125923   
outliers target_diff  XGB            0.002955        0.001387        0.084199   

                                 test_f1_std  test_prec_std  test_rec_std  \
preproc  target       metamodel                                             
standard absquantile2 XGB           0.112157       0.124813      0.108027   
minmax   absquantile2 XGB           0.134994       0.151281      0.131582   
standard absquantile2 LogReg        0.124596       0.134411      0.120154   
                      MLP           0.129268       0.134498      0.125893   
outliers absquantile2 XGB           0.123134       0.147453      0.120090   
...                                      ...            ...           ...   
power    absquantile3 LogReg        0.094153       0.111062      0.093656   
outliers target_diff  LogReg        0.082490       0.117928      0.092393   
power    absquantile3 MLP           0.104604       0.130054      0.102715   
standard absquantile3 MLP           0.129115       0.144013      0.127325   
outliers target_diff  XGB           0.076503       0.088967      0.083354   

                                 test_roc_std  
preproc  target 

In [38]:
all_dfs.groupby(["preproc", "target"]).mean().sort_values(by="test_b_acc_mean", ascending=False, inplace=False)

test_b_acc_mean  test_f1_mean  test_prec_mean  \
preproc  target                                                              
standard absquantile2               0.528869      0.516676        0.533122   
outliers absquantile2               0.509821      0.477540        0.511476   
power    absuniform2                0.502571      0.753784        0.741528   
outliers absuniform2                0.500425      0.765198        0.739809   
standard rel_xgboost_resnet         0.500317      0.556946        0.565544   
         absuniform2                0.499438      0.764599        0.739213   
outliers rel_xgboost_resnet         0.494149      0.543011        0.539589   
minmax   absuniform2                0.494108      0.772852        0.729303   
         rel_xgboost_resnet         0.488015      0.548410        0.529987   
power    rel_xgboost_resnet         0.484705      0.545470        0.546607   
minmax   absquantile2               0.473909      0.458868        0.472653   
power    absquantile2               0.471726      0.455691        0.470929   
standard absuniform3                0.376943      0.596591        0.564874   
power    target_diff                0.375589      0.492453        0.485399   
standard target_diff                0.356065      0.466599        0.462236   
minmax   absuniform3                0.352902      0.591578        0.523943   
         target_diff                0.344087      0.436071        0.407882   
outliers absuniform3                0.343388      0.574173        0.529048   
power    absuniform3                0.342254      0.570213        0.524067   
standard absquantile3               0.340000      0.331030        0.350326   
outliers absquantile3               0.337130      0.295974        0.304600   
minmax   absquantile3               0.324907      0.312630        0.339489   
power    absquantile3               0.315741      0.299682        0.312372   
outliers target_diff                0.305384      0.372791        0.337992   

                             test_rec_mean  test_roc_mean  fit_time_std  \
preproc  target                                                           
standard absquantile2             0.528175       0.528869      0.001854   
outliers absquantile2             0.509021       0.509821      0.014819   
power    absuniform2              0.782090       0.502571      0.001563   
outliers absuniform2              0.802963       0.500425      0.013962   
standard rel_xgboost_resnet       0.585159       0.500317      0.005294   
         absuniform2              0.801720       0.499438      0.001223   
outliers rel_xgboost_resnet       0.593915       0.494149      0.015904   
minmax   absuniform2              0.825132       0.494108      0.016072   
         rel_xgboost_resnet       0.601270       0.488015      0.011765   
power    rel_xgboost_resnet       0.572566       0.484705      0.002064   
minmax   absquantile2             0.473968       0.473909      0.031620   
power    absquantile2             0.470106       0.471726      0.005565   
standard absuniform3              0.657169       0.541667      0.004044   
power    target_diff              0.523016       0.395833      0.001319   
standard target_diff              0.506190       0.451389      0.008176   
minmax   absuniform3              0.688439       0.487795      0.009282   
         target_diff              0.512381       0.486111      0.003409   
outliers absuniform3              0.642222       0.479377      0.013991   
power    absuniform3              0.637857       0.477694      0.005112   
standard absquantile3             0.340608            NaN      0.002254   
outliers absquantile3             0.338122            NaN      0.018969   
minmax   absquantile3             0.325397            NaN      0.010151   
power    absquantile3             0.315820            NaN      0.007465   
outliers target_diff              0.451111       0.440972      0.018273   

                             score_time_std  test